# Project Name: Balanced Tree Clothing Company Data Tranasfer Mysql Database to Google Bigquery for Analysis Purpose.

Introduction:Balanced Tree Clothing Company prides themselves on providing an optimised range of clothing and lifestyle wear for the modern adventurer!

Danny, the CEO of this trendy fashion company has asked you to assist the team’s merchandising teams analyse their sales performance and generate a basic financial report to share with the wider business.

Available Data: 
For this case study there is a total of 4 datasets for this case study - however you will only need to utilise 2 main tables to solve all of the regular questions, and the additional 2 tables are used only for the bonus challenge question!

# Import library:

In [1]:
import numpy as np
import pandas as pd
import pymysql
import mysql.connector

# MySql Database Connection 

In [2]:
cnx=mysql.connector.connect(user='root',password='root',host='localhost',database='clothing_company')

# Data Query and Some Data Tranasform

### product_details table

In [ ]:
query1=""" select * from product_details"""

In [ ]:
product_details=pd.read_sql(query1,cnx)

In [13]:
product_details.head(5)

,product_id,price,product_name,category_id,segment_id,style_id,category_name,segment_name,style_name
0,c4a632,13,Navy Oversized Jeans - Womens,1,3,7,Womens,Jeans,Navy Oversized
1,e83aa3,32,Black Straight Jeans - Womens,1,3,8,Womens,Jeans,Black Straight
2,e31d39,10,Cream Relaxed Jeans - Womens,1,3,9,Womens,Jeans,Cream Relaxed
3,d5e9a6,23,Khaki Suit Jacket - Womens,1,4,10,Womens,Jacket,Khaki Suit
4,72f5d4,19,Indigo Rain Jacket - Womens,1,4,11,Womens,Jacket,Indigo Rain


In [17]:
product_details.duplicated().sum()

0

In [18]:
product_details.isnull().sum()

product_id       0
price            0
product_name     0
category_id      0
segment_id       0
style_id         0
category_name    0
segment_name     0
style_name       0
dtype: int64

### product_hierarchy table

In [ ]:
query2=""" select * from product_hierarchy"""

In [ ]:
product_hierarchy=pd.read_sql(query2,cnx)

In [14]:
product_hierarchy.head(5)

,id,parent_id,level_text,level_name
0,1,NaN,Womens,Category
1,2,NaN,Mens,Category
2,3,1.0,Jeans,Segment
3,4,1.0,Jacket,Segment
4,5,2.0,Shirt,Segment


In [19]:
product_hierarchy.duplicated().sum()

0

In [20]:
product_hierarchy.isnull().sum()

id            0
parent_id     2
level_text    0
level_name    0
dtype: int64

In [23]:
product_hierarchy=product_hierarchy.dropna()

In [24]:
product_hierarchy.isnull().sum()

id            0
parent_id     0
level_text    0
level_name    0
dtype: int64

### product_prices table

In [ ]:
query3=""" select * from product_prices"""

In [ ]:
product_prices=pd.read_sql(query3,cnx)

In [15]:
product_prices.head(5)

,id,product_id,price
0,7,c4a632,13
1,8,e83aa3,32
2,9,e31d39,10
3,10,d5e9a6,23
4,11,72f5d4,19


In [25]:
product_prices.duplicated().sum()

0

In [26]:
product_prices.isnull().sum()

id            0
product_id    0
price         0
dtype: int64

### sales table

In [ ]:
query4=""" select * from saless"""

In [ ]:
sales=pd.read_sql(query4,cnx)

In [16]:
sales.head(5)

,prod_id,qty,price,discount,member,txn_id,start_txn_time
0,c4a632,4,13,17,t,54f307,2021-02-13 01:59:43
1,5d267b,4,40,17,t,54f307,2021-02-13 01:59:43
2,b9a74d,4,17,17,t,54f307,2021-02-13 01:59:43
3,2feb6b,2,29,17,t,54f307,2021-02-13 01:59:43
4,c4a632,5,13,21,t,26cc98,2021-01-19 01:39:00


In [27]:
sales.duplicated().sum()

0

In [28]:
sales.isnull().sum()

prod_id           0
qty               0
price             0
discount          0
member            0
txn_id            0
start_txn_time    0
dtype: int64

In [29]:
sales['start_txn_time']=pd.to_datetime(sales['start_txn_time'])

In [34]:
sales['Year']=sales['start_txn_time'].dt.year
sales['month']=sales['start_txn_time'].dt.month
sales['month_name']=sales['start_txn_time'].dt.month_name()
sales['Day']=sales['start_txn_time'].dt.day
sales['Day_Name']=sales['start_txn_time'].dt.day_name()
sales['Hour']=sales['start_txn_time'].dt.hour

In [36]:
sales['revenue']=sales['qty']*sales['price']

In [39]:
sales.rename(columns={'prod_id':'product_id'},inplace=True)

In [40]:
sales.head(5)

,product_id,qty,price,discount,member,txn_id,start_txn_time,Year,month,month_name,Day,Day_Name,Hour,revenue
0,c4a632,4,13,17,t,54f307,2021-02-13 01:59:43,2021,2,February,13,Saturday,1,52
1,5d267b,4,40,17,t,54f307,2021-02-13 01:59:43,2021,2,February,13,Saturday,1,160
2,b9a74d,4,17,17,t,54f307,2021-02-13 01:59:43,2021,2,February,13,Saturday,1,68
3,2feb6b,2,29,17,t,54f307,2021-02-13 01:59:43,2021,2,February,13,Saturday,1,58
4,c4a632,5,13,21,t,26cc98,2021-01-19 01:39:00,2021,1,January,19,Tuesday,1,65


## Merge Data 

In [41]:
df1=pd.merge(product_details,sales,how='left',on='product_id')

# Export Data Google Bigquery

In [43]:
from google.cloud import bigquery
from pandas_gbq import to_gbq

In [44]:
project_id='store-analysis-399009'
dataset_id='etl_clothing'
table_id='product_details'

In [46]:
to_gbq(product_details, destination_table=f"{project_id}.{dataset_id}.{table_id}", project_id=project_id, if_exists='replace')

100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1000.79it/s]


In [ ]:
project_id='store-analysis-399009'
dataset_id='etl_clothing'
table_id='product_hierarchy'

In [47]:
to_gbq(product_hierarchy, destination_table=f"{project_id}.{dataset_id}.{table_id}", project_id=project_id, if_exists='replace')

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


In [48]:
project_id='store-analysis-399009'
dataset_id='etl_clothing'
table_id='product_prices'

In [49]:
to_gbq(product_prices, destination_table=f"{project_id}.{dataset_id}.{table_id}", project_id=project_id, if_exists='replace')

100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1002.94it/s]


In [51]:
project_id='store-analysis-399009'
dataset_id='etl_clothing'
table_id='sales'

In [52]:
to_gbq(sales, destination_table=f"{project_id}.{dataset_id}.{table_id}", project_id=project_id, if_exists='replace')

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]
